# Julia Q&A

## Q: How can I ensure that the radius of a circle is positive?

In [ ]:
abstract type Shape end

"""
Defines a point in a 2D space, where x and y are the coordinates of the point on the x-axis and y-axis, respectively.
""" 
struct Point
    x::Real
    y::Real
end


"""
Defines a circle in a 2D space, where center is the center of the circle and radius is the radius of the circle.
"""
struct Circle <: Shape
    center::Point
    radius::Real

    # Inner constructor for Circle
    function Circle(center::Point, radius::Real)
        if radius <= 0
            error("Radius must be positive")
        end
        new(center, radius)
    end

end

In [ ]:
Circle(Point(0,0), 1)

In [ ]:
Circle(Point(0,0), -1)

## Q: How can I define a custom string representation for my `struct`?

I want to override the default representation and define how the struct is rendered as text.

In [ ]:
struct Person
    first_name::String
    last_name::String
    age::Int
end



The default representation:

In [ ]:
Person("John", "Doe", 30)

In order to override it, define the following function for the custom type:

In [ ]:
# Overload the `show` function for the `Person` type
function Base.show(io::IO, person::Person)
    print(io, "$(person.first_name) $(person.last_name), age $(person.age)")
end



In [ ]:
jane_doe = Person("Jane", "Doe", 30)


In [ ]:
println(jane_doe) 

## How can I modify a dataframe column inplace?

In [ ]:
import Pkg
Pkg.add("DataFrames") # data frames
Pkg.add("CSV")  # reading csv files

In [ ]:
using DataFrames, CSV

In [ ]:
data = CSV.read(joinpath("../.assets/data/", "houses_seattle", "kc_house_data.csv"), DataFrame)

Multiplying a column with a factor like this...

In [ ]:
data[!, :price] .* 2

... does not modify the original dataframe:

In [ ]:
data[!, [:price]]

To modify the original dataframe object, use either...

In [ ]:
data[!, :price] = data[!, :price] .* 2

In [ ]:
data[:, :price] = data[:, :price] .* 2

In [ ]:
data[!, [:price]]

...or...

In [ ]:
transform!(
    data, 
    :price => (x -> 2 .* x) => :price
)


In [ ]:
data

Without the !, the function does not modify the original dataframe inplace:

In [ ]:
transform(
    data, 
    :price => (x -> 2 .* x) => :price
)


In [ ]:
data

## Q: How to install a specific version of a package?

In [ ]:
using Pkg


In [ ]:
Pkg.rm("Example")

In [ ]:
Pkg.add(
    PackageSpec(
        name="Example", 
        version="0.5.0"
    )
)

## Q: Can I use Sparse Matrices?

In [ ]:
using SparseArrays


In [ ]:
@doc sparse

In [ ]:

# Create a sparse matrix
# SparseArrays.sparse(I, J, V, m, n) creates a sparse matrix with elements V at
# positions (I[k], J[k]) for k in 1:length(V), with overall size m x n.
I = [1, 2, 4]  # Row indices
J = [1, 2, 3]  # Column indices
V = [10, 20, 30]  # Values at the above positions
A = sparse(I, J, V, 4, 4)  # 4x4 sparse matrix


Sparse matrix operations are basically the same as dense matrix operations:

In [ ]:
B = A + A  # Add two sparse matrices


In [ ]:
C = A * 2  # Scalar multiplication


In [ ]:
D = A'  # Transpose of the sparse matrix


In [ ]:
x = [1, 0, 0, 0]
y = A * x  # Matrix-vector multiplication

## Q: Does Julia pass function arguments by value or by reference?

_Neither!_

The strategy used is _call by sharing_, and it is the same strategy used in other languages such as Python. When an argument is passed to a function, both the caller and the callee have access to the same data object (they "share" the argument's data). 



In [13]:
function modify_array!(arr)
    arr[1] = 99
end

modify_array! (generic function with 1 method)

In [14]:
x = [1,2,3]

modify_array!(x)

x

3-element Vector{Int64}:
 99
  2
  3

If I want "call by value" behavior, I have to make an explicit copy of a (mutable) argument:

In [15]:
function modify_array_copy(arr)
    arr = copy(arr) # Explicitly create a copy of the array
    arr[1] = 99
    return arr
end

modify_array_copy (generic function with 1 method)

In [16]:
x = [1,2,3]

x_mod = modify_array_copy(x)

x

3-element Vector{Int64}:
 1
 2
 3

In [17]:
x_mod

3-element Vector{Int64}:
 99
  2
  3

_Why is this not "call by reference"?_

If you reassign the parameter within the function to point to a different object, this change will not affect the original reference in the caller's scope. In true "call by reference", reassigning the reference would also change the original variable's reference in the caller's context.

In [18]:
function rebind_array(arr)
    arr = [99, 99, 99] # This creates a new local array and binds `arr` to it
    return arr
end

rebind_array (generic function with 1 method)

In [19]:
x = [1,2,3]

rebind_array(x)

x

3-element Vector{Int64}:
 1
 2
 3

# Q: Views of Arrays

---
_This notebook is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/). Copyright © 2018-2024 [Point 8 GmbH](https://point-8.de)_^m